<a href="https://colab.research.google.com/github/liuxinming666/RNN/blob/master/%E5%8A%A0%E8%BD%BD%E9%A2%84%E8%AE%AD%E7%BB%83%E5%AD%97%E5%90%91%E9%87%8F%EF%BC%8C%E6%96%87%E6%9C%AC%E7%94%9F%E6%88%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 导入类库

In [0]:
import collections
import time
import numpy as np
import random
import math

try:
  # Use the %tensorflow_version magic if in colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf

# 加载预训练字向量

In [0]:
start = time.clock()
charFilePath = '/content/drive/My Drive/Colab Notebooks/RNN/预训练词向量/sgns.literature.char'
f = open(charFilePath)
max_chars = 0  #预训练字向量的个数
embedding_dim = 0   #每个字的维度为300维

embedding_index = {}
bRead = False
for line in f:
  if bRead == False:
    values = line.split(' ',1)
    max_chars = values[0]
    embedding_dim = values[1]
  if bRead == True:
    values = line.split(' ',1)
    char = values[0]
    charAry = values[1].split(' ')
    charAry = charAry[:300]
    coefs = np.asarray(charAry,dtype=np.float32)
    embedding_index[char] = coefs
  bRead = True

f.close()
end = time.clock()
print('处理用时:',end - start)

# 古诗数据处理

In [0]:
#读取古诗,将古诗存在数组中
poetry_file = '/content/drive/My Drive/数据集/古诗文本/poetry.txt'
# 诗集  
poetrys = []
filterSAry = ["_", "(", "《","["]
index = 0  
with open(poetry_file, "r") as f:  
    for line in f: 
      line = line.strip('\n')
      lineAry = line.strip(' ').split(':')
      title = lineAry[0]
      content = lineAry[1]

      if '_' in content or '（' in content or '《' in content or '[' in content:
        continue
      if len(content) < 5 or len(content) > 79:
        continue
      
      content = content + ']'
      poetrys.append(content)

In [0]:
#按诗的字数排序
poetrys = sorted(poetrys,key=lambda poetry:len(poetry))
print('诗的总数为:',len(poetrys))

In [0]:
#统计每首诗的长度,并统计相同长度的诗都各有多少首，统计出哪种长度的诗个数最多
poetrys_length = [len(poetry) for poetry in poetrys]
counter = collections.Counter(poetrys_length)
poetrys_length,poetrys_num = zip(*counter.items())
#求出最多个数的索引
poertyLength_maxNum_index = np.argmax(poetrys_num)
#print(counter)
print('数量最多的诗的长度为:',poetrys_length[poertyLength_maxNum_index])
print('数量最多的诗的个数为:',poetrys_num[poertyLength_maxNum_index])

In [0]:
#将数量最多的诗装进列表
poetrys_train = [poetry for poetry in poetrys if len(poetry) == 49]
print('将要参与训练的诗的个数为:',len(poetrys_train))

In [0]:
# 统计每个字出现次数
start = time.clock()
all_words = []
for poetry in poetrys_train:
  all_words += [word for word in poetry]
#words = list(set(all_words))
words_count = collections.Counter(all_words)
words_count = sorted(
    words_count.items(), 
    key=lambda x: x[1],
    reverse=True)
words,_ = zip(*words_count)
end = time.clock()
print('一共有',len(words),'个字符')
print('用时:',end - start)
print(words[:10])
#print(list(set(result)).sort(key=result.)) 

# 构造训练数据集

In [0]:
#定义初始变量，文本的长度为49,参与训练的样本为12530个
train_num = 12530
maxlen = 49

In [0]:
#将字符映射为数组，并将数字映射为字符
char_to_ix = {ch:i+1 for i,ch in enumerate(words)}
ix_to_char = {i+1:ch for i,ch in enumerate(words)}

In [0]:
#构造一个生成器
#x shape为 (batch,maxlen - 1)
#y shape为 (batch,maxlen - 1)
def gen_generator():
  dataLength = len(poetrys_train)
  batch = 32
  while True:
    random.shuffle(poetrys_train)
    data_batch = np.zeros((32, maxlen))
    for i in range(dataLength):
      for wix,word in enumerate(poetrys_train[i]):
        data_batch[i%batch,wix] = char_to_ix[word]
      if (i%(batch-1)) == 0 and i != 0:
        data_baatch_oneHot = tf.keras.utils.to_categorical(data_batch,num_classes=len(words) + 1)
        x = data_batch[:,:maxlen - 1]
        y = data_baatch_oneHot[:,1:]
        yield (x,y)


gen = gen_generator()
A = next(gen)
print(A[0].shape)
print(A[0][0][0])
print(poetrys_train[0][0])
print(char_to_ix[poetrys_train[0][0]])


# 准备嵌入层

In [0]:
#构造嵌入矩阵
embedding_matrix = np.zeros((len(words) + 1, int(embedding_dim)))
for char,index in char_to_ix.items():
  embedding_vector = embedding_index.get(char)
  if embedding_vector is not None:
    embedding_matrix[index] = embedding_vector

In [0]:
'''
#构造嵌入层
embedding_layer = tf.keras.layers.Embedding(len(words) + 1,
                      int(embedding_dim),
                      weights=[embedding_matrix],
                      input_length=maxlen - 1,
                      trainable=False)
'''

# 构造模型

In [0]:
#构造模型
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(input_dim=len(words) + 1,output_dim=int(embedding_dim),input_length=maxlen - 1))
model.add(tf.keras.layers.LSTM(
    1024,
    return_sequences = True))
#model.add(tf.keras.layers.LSTM(128,return_sequences = True))
model.add(tf.keras.layers.Dense(len(words) + 1,activation='softmax'))
#model.add(tf.keras.layers.LSTM(len(words) + 1))

model.compile(loss='categorical_crossentropy',optimizer='adam')

In [0]:
#设置嵌入层参数
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

In [0]:
model.summary()

# 训练模型

In [0]:
#训练模型
gen = gen_generator()
train_epochs = 100
stepsPerEpochs = math.floor(len(poetrys_train)/32)
model.fit_generator(gen,
          epochs=train_epochs,
          steps_per_epoch=stepsPerEpochs)

In [0]:
#模型保存
modelPath = '/content/drive/My Drive/myModel/rnn-poetrys-embedding-100.h5'
model.save(modelPath)

# 加载训练好的模型进行测试

In [0]:
#加载模型
modelPath = '/content/drive/My Drive/myModel/rnn-poetrys-embedding-100.h5'
model = tf.keras.models.load_model(modelPath)
model.summary()

In [0]:
#计算出第一个字是什么
#x shape为 (batch,maxlen - 1)
#y shape为 (batch,maxlen - 1,len(words)+1)
x = np.zeros(shape=(1,maxlen - 1))
y = model.predict(x)

firstWord_oneHot = y[0,0,:]
print(firstWord_oneHot.shape)
idx = np.argmax(firstWord_oneHot,axis=-1)
firstWord = ix_to_char[idx]
print(firstWord)


In [0]:
print(firstWord_oneHot.shape)

In [0]:
#循环得出每一个字
genPoetry = []
genPoetry.append(firstWord)
xPoetry = np.zeros(shape=(1,maxlen - 1))
xPoetry[0,0] = char_to_ix[firstWord]
#print(np.argmax(xPoetry[0,0]))
for i in range(maxlen - 2):
  y = model.predict(xPoetry)
  if i < (maxlen -1):
    xPoetry[0,i+1] = np.argmax(y[0,i,:],axis=-1)
  idxTmp = np.argmax(y[0,i,:],axis=-1)
  genPoetry.append(ix_to_char[idxTmp])

In [0]:
print("生成的诗为：")
print("".join(genPoetry))